### Download and Store Data Files

In [1]:
import os
import ssl
import sys
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', None)

#### Downloading the Files

In [2]:
gradurls = {'https://infohub.nyced.org/docs/default-source/default-document-library/2020-graduation_rates_public_school.xlsx': 'grad_rates.xlsx',
            'https://infohub.nyced.org/docs/default-source/default-document-library/2020-graduation_rates_public_charters.xlsx': 'charter_grad_rates.xlsx'}
sqrurls = {'https://infohub.nyced.org/docs/default-source/default-document-library/201920_hs_sqr_results.xlsx': 'sqr2020.xlsx',
           'https://infohub.nyced.org/docs/default-source/default-document-library/201819_hs_sqr_results.xlsx': 'sqr2019.xlsx',
           'https://infohub.nyced.org/docs/default-source/default-document-library/201718_hs_sqr_results.xlsx': 'sqr2018.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2016-17_hs_sqr.xlsx': 'sqr2017.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2015_2016_hs_sqr_results_2017_01_05.xlsx': 'sqr2016.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2014_2015_hs_sqr_results_2016_04_08.xlsx': 'sqr2015.xlsx'}
demourls = {'https://infohub.nyced.org/docs/default-source/default-document-library/demographic-snapshot-2015-16-to-2019-20-(public).xlsx': 'demo.xlsx'}
urls = [gradurls, sqrurls, demourls]

In [3]:
for url in urls:
    try:
        [urllib.request.urlretrieve(x, 'data/'+url[x]) for x in url]
    except:
        print("Trying the next url")

Trying the next url
Trying the next url
Trying the next url


#### Graduation Rate Files

In [4]:
gradrates = pd.read_excel('data/grad_rates.xlsx', sheet_name = "All")
charterrates = pd.read_excel('data/charter_grad_rates.xlsx', sheet_name = "All")

In [5]:
gradrates[['DBN', 'School Name', 'Cohort Year', 'Cohort', '# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']].head()

,DBN,School Name,Cohort Year,Cohort,# Total Cohort,# Grads,# Dropout,# Still Enrolled
0,01M292,ORCHARD COLLEGIATE ACADEMY,2016,4 year August,31,29,1,1
1,01M292,ORCHARD COLLEGIATE ACADEMY,2015,4 year August,28,25,3,0
2,01M292,ORCHARD COLLEGIATE ACADEMY,2014,4 year August,29,28,1,0
3,01M292,ORCHARD COLLEGIATE ACADEMY,2013,4 year August,36,25,7,3
4,01M292,ORCHARD COLLEGIATE ACADEMY,2012,4 year August,44,24,10,10


In [6]:
charterrates[['DBN', 'School Name', 'Cohort Year', '# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']].head()

,DBN,School Name,Cohort Year,# Total Cohort,# Grads,# Dropout,# Still Enrolled
0,84K355,WILLIAMSBURG COLLEGIATE CHARTER SCHOOL,2012,37,34,0,3
1,84K355,WILLIAMSBURG COLLEGIATE CHARTER SCHOOL,2011,37,28,1,8
2,84K355,WILLIAMSBURG COLLEGIATE CHARTER SCHOOL,2010,31,26,0,5
3,84K355,WILLIAMSBURG COLLEGIATE CHARTER SCHOOL,2009,31,20,0,3
4,84K355,WILLIAMSBURG COLLEGIATE CHARTER SCHOOL,2012,37,34,0,3


In [7]:
all_grad_rates = pd.concat([gradrates[['DBN', 'School Name', 'Cohort Year', 'Cohort', '# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']],
                            charterrates[['DBN', 'School Name', 'Cohort Year', 'Cohort', '# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']]], 
                           ignore_index=True)
all_grad_rates[['# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']] = all_grad_rates[['# Total Cohort', '# Grads', '# Dropout', '# Still Enrolled']].apply(pd.to_numeric, errors='coerce')

In [8]:
all_grad_rates.rename(columns={'DBN':'dbn',
                               'School Name':'schoolName',
                               'Cohort Year':'cohortYear', 
                               'Cohort':'cohort', 
                               '# Total Cohort':'totalCohortNum',
                               '# Grads':'gradsNum', '# Dropout':'dropoutNum', '# Still Enrolled':'stillEnrolledNum'}, inplace=True)

In [9]:
all_grad_rates.to_csv('data/clean/all_grad_rates.csv',index=False)

In [10]:
all_grad_rates.head()

,dbn,schoolName,cohortYear,cohort,totalCohortNum,gradsNum,dropoutNum,stillEnrolledNum
0,01M292,ORCHARD COLLEGIATE ACADEMY,2016,4 year August,31,29.0,1.0,1.0
1,01M292,ORCHARD COLLEGIATE ACADEMY,2015,4 year August,28,25.0,3.0,0.0
2,01M292,ORCHARD COLLEGIATE ACADEMY,2014,4 year August,29,28.0,1.0,0.0
3,01M292,ORCHARD COLLEGIATE ACADEMY,2013,4 year August,36,25.0,7.0,3.0
4,01M292,ORCHARD COLLEGIATE ACADEMY,2012,4 year August,44,24.0,10.0,10.0


#### SQR Files

In [11]:
tabs = ['Summary', 'Student Achievement', 'Closing the Achievement Gap', 'Framework', 'Additional Info']
sqr_files = ['sqr2020.xlsx', 'sqr2019.xlsx', 'sqr2018.xlsx', 'sqr2017.xlsx', 'sqr2016.xlsx', 'sqr2015.xlsx']

In [12]:
summary = {}
stud_achieve = {}
closing_gap = {}
framework = {}
add_info = {}

for f in sqr_files:
    summary[f] = pd.read_excel('data/'+f, sheet_name='Summary')
    stud_achieve[f] = pd.read_excel('data/'+f, sheet_name='Student Achievement')
    closing_gap[f] = pd.read_excel('data/'+f, sheet_name='Closing the Achievement Gap')
    framework[f] = pd.read_excel('data/'+f, sheet_name='Framework')
    add_info[f] = pd.read_excel('data/'+f, sheet_name='Additional Info')

In [13]:
summary['sqr2020.xlsx'].columns = summary['sqr2020.xlsx'].iloc[2]
summary['sqr2019.xlsx'].columns = summary['sqr2019.xlsx'].iloc[2]
summary['sqr2018.xlsx'].columns = summary['sqr2018.xlsx'].iloc[0]
summary['sqr2017.xlsx'].columns = summary['sqr2017.xlsx'].iloc[0]
summary['sqr2016.xlsx'].columns = summary['sqr2016.xlsx'].iloc[0]
summary['sqr2015.xlsx'].columns = summary['sqr2015.xlsx'].iloc[0]

In [14]:
summary['sqr2020.xlsx'].columns

Index([                                                                                                           nan,
       'Due to COVID-19, Framework ratings and scores are not available for any schools for the 2019-20 school year.',
                                                                                                                  nan,
                                                                                                                'DBN',
                                                                                                        'School Name',
                                                                                                        'School Type',
                                                                                                         'Enrollment',
                                                                            'Rigorous Instruction - Percent Positive',
                                                

In [15]:
summary_cols = ['DBN','Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']
summary['sqr2019.xlsx'] = summary['sqr2019.xlsx'][summary_cols]
summary['sqr2018.xlsx'] = summary['sqr2018.xlsx'][summary_cols]
summary['sqr2017.xlsx'] = summary['sqr2017.xlsx'][summary_cols]
summary['sqr2016.xlsx'] = summary['sqr2016.xlsx'][summary_cols]
summary['sqr2015.xlsx'] = summary['sqr2015.xlsx'][summary_cols]

In [16]:
summary['sqr2020.xlsx'] = summary['sqr2020.xlsx'][['DBN','Enrollment',
                         'Student Percent - Asian','Student Percent - Black','Student Percent - Hispanic',
                         'Student Percent - White', 'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                         'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                         'Average Student Attendance (through Feb-2020 only)',
                         'Percent of Students Chronically Absent (through Feb-2020 only)']]

In [17]:
summary['sqr2020.xlsx'].rename(columns={'Student Percent - Asian':'Percent Asian',
                                        'Student Percent - Black':'Percent Black',
                                        'Student Percent - Hispanic': 'Percent Hispanic',
                                        'Student Percent - White': 'Percent White', 
                                        'Average Student Attendance (through Feb-2020 only)' : 'Student Attendance Rate',
                                        'Percent of Students Chronically Absent (through Feb-2020 only)':'Percent of Students Chronically Absent'}, inplace=True)

In [18]:
summary['sqr2020.xlsx']['SQR Year'] = '2020'
summary['sqr2019.xlsx']['SQR Year'] = '2019'
summary['sqr2018.xlsx']['SQR Year'] = '2018'
summary['sqr2017.xlsx']['SQR Year'] = '2017'
summary['sqr2016.xlsx']['SQR Year'] = '2016'
summary['sqr2015.xlsx']['SQR Year'] = '2015'

In [19]:
summary_df = pd.concat(summary).reset_index().drop(['level_0', 'level_1'], axis = 1)
summary_df = summary_df[summary_df.DBN != 'DBN']

In [20]:
summary_df[['Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']] = summary_df[['Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']].apply(pd.to_numeric, errors='coerce')

In [21]:
stud_achieve['sqr2020.xlsx'].columns = stud_achieve['sqr2020.xlsx'].iloc[2]
stud_achieve['sqr2019.xlsx'].columns = stud_achieve['sqr2019.xlsx'].iloc[2]
stud_achieve['sqr2018.xlsx'].columns = stud_achieve['sqr2018.xlsx'].iloc[0]
stud_achieve['sqr2017.xlsx'].columns = stud_achieve['sqr2017.xlsx'].iloc[0]
stud_achieve['sqr2016.xlsx'].columns = stud_achieve['sqr2016.xlsx'].iloc[0]
stud_achieve['sqr2015.xlsx'].columns = stud_achieve['sqr2015.xlsx'].iloc[0]

In [22]:
sqrcols = ["DBN", "N count - 10+ Credits in 1st Year - All Students",
                              "Metric Value - 10+ Credits in 1st Year - All Students",
                              "N count - 10+ Credits in 1st Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                              "N count - 10+ Credits in 2nd Year - All Students",
                              "Metric Value - 10+ Credits in 2nd Year - All Students",
                              "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                              "N count - 10+ Credits in 3rd Year - All Students",
                              "Metric Value - 10+ Credits in 3rd Year - All Students",
                              "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]
stud_achieve['sqr2020.xlsx'] = stud_achieve['sqr2020.xlsx'][sqrcols]
stud_achieve['sqr2019.xlsx'] = stud_achieve['sqr2019.xlsx'][sqrcols]
stud_achieve['sqr2018.xlsx'] = stud_achieve['sqr2018.xlsx'][sqrcols]
stud_achieve['sqr2017.xlsx'] = stud_achieve['sqr2017.xlsx'][sqrcols]

sqrcols = ["DBN", "Metric Value - Percentage Earning 10+ Credits in First Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in First Year",
           "Metric Value - Percentage Earning 10+ Credits in Second Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Second Year",
           "Metric Value - Percentage Earning 10+ Credits in Third Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Third Year"]
stud_achieve['sqr2015.xlsx'] = stud_achieve['sqr2015.xlsx'][sqrcols]
stud_achieve['sqr2016.xlsx'] = stud_achieve['sqr2016.xlsx'][sqrcols]

In [23]:
sqrcols = {"Metric Value - Percentage Earning 10+ Credits in First Year":"Metric Value - 10+ Credits in 1st Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in First Year": "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
           "Metric Value - Percentage Earning 10+ Credits in Second Year" : "Metric Value - 10+ Credits in 2nd Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Second Year": "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
           "Metric Value - Percentage Earning 10+ Credits in Third Year": "Metric Value - 10+ Credits in 3rd Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Third Year": "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"}
stud_achieve['sqr2015.xlsx'].rename(columns = sqrcols, inplace=True)
stud_achieve['sqr2016.xlsx'].rename(columns = sqrcols, inplace=True)

In [24]:
stud_achieve['sqr2020.xlsx']['SQR Year'] = '2020'
stud_achieve['sqr2019.xlsx']['SQR Year'] = '2019'
stud_achieve['sqr2018.xlsx']['SQR Year'] = '2018'
stud_achieve['sqr2017.xlsx']['SQR Year'] = '2017'
stud_achieve['sqr2016.xlsx']['SQR Year'] = '2016'
stud_achieve['sqr2015.xlsx']['SQR Year'] = '2015'

In [25]:
stud_achieve_df = pd.concat(stud_achieve).reset_index().drop(['level_0', 'level_1'], axis=1)
stud_achieve_df = stud_achieve_df[stud_achieve_df.DBN != 'DBN']

/Users/mguan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
stud_achieve_df[["N count - 10+ Credits in 1st Year - All Students", "Metric Value - 10+ Credits in 1st Year - All Students",
                 "N count - 10+ Credits in 1st Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                 "N count - 10+ Credits in 2nd Year - All Students",
                 "Metric Value - 10+ Credits in 2nd Year - All Students",
                 "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                 "N count - 10+ Credits in 3rd Year - All Students",
                 "Metric Value - 10+ Credits in 3rd Year - All Students",
                 "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]] = stud_achieve_df[["N count - 10+ Credits in 1st Year - All Students",
                              "Metric Value - 10+ Credits in 1st Year - All Students",
                              "N count - 10+ Credits in 1st Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                              "N count - 10+ Credits in 2nd Year - All Students",
                              "Metric Value - 10+ Credits in 2nd Year - All Students",
                              "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                              "N count - 10+ Credits in 3rd Year - All Students",
                              "Metric Value - 10+ Credits in 3rd Year - All Students",
                              "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]].apply(pd.to_numeric, errors='coerce')

In [27]:
closing_gap['sqr2020.xlsx'].columns = closing_gap['sqr2020.xlsx'].iloc[2]
closing_gap['sqr2019.xlsx'].columns = closing_gap['sqr2019.xlsx'].iloc[2]
closing_gap['sqr2018.xlsx'].columns = closing_gap['sqr2018.xlsx'].iloc[0]
closing_gap['sqr2017.xlsx'].columns = closing_gap['sqr2017.xlsx'].iloc[0]
closing_gap['sqr2016.xlsx'].columns = closing_gap['sqr2016.xlsx'].iloc[0]
closing_gap['sqr2015.xlsx'].columns = closing_gap['sqr2015.xlsx'].iloc[0]

closing_gap['sqr2020.xlsx']['sqrYear'] = '2020'
closing_gap['sqr2019.xlsx']['sqrYear'] = '2019'
closing_gap['sqr2018.xlsx']['sqrYear'] = '2018'
closing_gap['sqr2017.xlsx']['sqrYear'] = '2017'
closing_gap['sqr2016.xlsx']['sqrYear'] = '2016'
closing_gap['sqr2015.xlsx']['sqrYear'] = '2015'

In [28]:
closing_gap['sqr2020.xlsx'] = closing_gap['sqr2020.xlsx'][['DBN', 'N count - 4-Year Graduation Rate - Lowest Third Citywide',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide', 'sqrYear']]
closing_gap['sqr2020.xlsx'].rename(columns={'DBN':'dbn',
                                            'N count - 4-Year Graduation Rate - Lowest Third Citywide': 'numGradLowest3rd',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide': 'pctGradLowest3rd',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)': 'numCollegeReadyLowest3rd',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)': 'pctCollegeReadyLowest3rd',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'numPERLowest3rd' ,
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'pctPERLowest3rd',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide': 'numCCPCILowest3rd',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide': 'pctCCPCILowest3rd'}, inplace=True)

In [29]:
closing_gap_cols = ['DBN', 'N count - 4-Year Graduation Rate - Lowest Third Citywide',
                    'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide',
                    'N count - 4-Year College Readiness Index, Lowest Third Citywide',
                    'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide',
                    'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                    'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                    'N count - College and Career Preparatory Course Index, Lowest Third Citywide',
                    'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide', 'sqrYear']
closing_gap['sqr2019.xlsx'] = closing_gap['sqr2019.xlsx'][closing_gap_cols]
closing_gap['sqr2018.xlsx'] = closing_gap['sqr2018.xlsx'][closing_gap_cols]
closing_gap['sqr2017.xlsx'] = closing_gap['sqr2017.xlsx'][closing_gap_cols]

In [30]:
temp = pd.concat([closing_gap['sqr2019.xlsx'], closing_gap['sqr2018.xlsx'], closing_gap['sqr2017.xlsx']], axis=0)
temp.rename(columns = {'DBN': 'dbn',
                       'N count - 4-Year Graduation Rate - Lowest Third Citywide': 'numGradLowest3rd',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide': 'pctGradLowest3rd',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide': 'numCollegeReadyLowest3rd',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide': 'pctCollegeReadyLowest3rd',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'numPERLowest3rd' ,
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'pctPERLowest3rd',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide': 'numCCPCILowest3rd',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide': 'pctCCPCILowest3rd'}, inplace=True)

In [31]:
closing_gap_cols = ['DBN', 'Metric Value - Graduation Rate, 4-year, lowest third city',
                    'Metric Value - Percentage in Lowest Third City, 4-year College Readiness Index',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index', 'sqrYear']

closing_gap['sqr2016.xlsx'] = closing_gap['sqr2016.xlsx'][closing_gap_cols]
closing_gap['sqr2016.xlsx'].rename(columns = {'DBN':'dbn',
                                              'Metric Value - Graduation Rate, 4-year, lowest third city':'pctGradLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, 4-year College Readiness Index':'pctCollegeReadyLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School':'pctPERLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index':'pctCCPCILowest3rd'}, inplace=True)

In [32]:
closing_gap_cols = ['DBN', 'Metric Value - Weighted Diploma Rate, 4 Year - Lowest Third City',
                    'Metric Value - Percentage in Lowest Third City, 4-year Non-Remediation Index',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index', 'sqrYear']

closing_gap['sqr2015.xlsx'] = closing_gap['sqr2015.xlsx'][closing_gap_cols]
closing_gap['sqr2015.xlsx'].rename(columns = {'DBN':'dbn',
                                              'Metric Value - Weighted Diploma Rate, 4 Year - Lowest Third City':'pctGradLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, 4-year Non-Remediation Index':'pctCollegeReadyLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School':'pctPERLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index':'pctCCPCILowest3rd'}, inplace=True)

In [33]:
closing_gap_df = pd.concat([closing_gap['sqr2020.xlsx'], temp, closing_gap['sqr2016.xlsx'],closing_gap['sqr2015.xlsx']], axis=0)

/Users/mguan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [34]:
closing_gap_df = closing_gap_df[closing_gap_df.dbn.str.len()==6]
closing_gap_df[['numCCPCILowest3rd', 'numCollegeReadyLowest3rd',
       'numGradLowest3rd', 'numPERLowest3rd', 'pctCCPCILowest3rd',
       'pctCollegeReadyLowest3rd', 'pctGradLowest3rd', 'pctPERLowest3rd']] = closing_gap_df[['numCCPCILowest3rd', 'numCollegeReadyLowest3rd',
       'numGradLowest3rd', 'numPERLowest3rd', 'pctCCPCILowest3rd',
       'pctCollegeReadyLowest3rd', 'pctGradLowest3rd', 'pctPERLowest3rd']].apply(pd.to_numeric, errors='coerce')

In [35]:
sqr_df = summary_df.merge(stud_achieve_df, how='inner', left_on = ['DBN', 'SQR Year'], right_on = ['DBN', 'SQR Year'])

In [36]:
sqr_df.drop(columns=['Percent Asian', 'Percent Black',
       'Percent Hispanic', 'Percent White',
       'Percent English Language Learners',
       'Percent Students with Disabilities', 'Percent Self-Contained',
       'Economic Need Index'], inplace=True)

In [37]:
sqr_df.rename(columns={"DBN":'dbn', 'Enrollment':'sqrEnrollment',
                       "Average Grade 8 English Proficiency": 'grd8Ela',
                       "Average Grade 8 Math Proficiency": 'grd8Math',
                       "Student Attendance Rate": 'attdRate',
                       "Percent of Students Chronically Absent" : 'pctChronAbs',
                       "SQR Year": 'sqrYear',
                       "Metric Value - 10+ Credits in 1st Year - All Students" : 'pct10PlusYear1',
                       "Metric Value - 10+ Credits in 1st Year - School's Lowest Third" : 'pct10PlusYear1Lowest3rd',
                       "Metric Value - 10+ Credits in 2nd Year - All Students" : 'pct10PlusYear2',
                       "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third" : 'pct10PlusYear2Lowest3rd',
                       "Metric Value - 10+ Credits in 3rd Year - All Students" : 'pct10PlusYear3',
                       "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third" : 'pct10PlusYear3Lowest3rd',
                       "N count - 10+ Credits in 1st Year - All Students" : 'num10PlusYear1',
                       "N count - 10+ Credits in 1st Year - School's Lowest Third" : 'num10PlusYear1Lowest3rd',
                       "N count - 10+ Credits in 2nd Year - All Students" : 'num10PlusYear2',
                       "N count - 10+ Credits in 2nd Year - School's Lowest Third" : 'num10PlusYear2Lowest3rd',
                       "N count - 10+ Credits in 3rd Year - All Students" : 'num10PlusYear3',
                       "N count - 10+ Credits in 3rd Year - School's Lowest Third": 'num10PlusYear3Lowest3rd'}, inplace=True)

back into the count of lowest 3 students by year for schools reporting student counts

In [38]:
sqr_df['numYear1Lowest3rdApprox'] = round(sqr_df['num10PlusYear1Lowest3rd'] / sqr_df['pct10PlusYear1Lowest3rd'],0)
sqr_df['numYear2Lowest3rdApprox'] = round(sqr_df['num10PlusYear2Lowest3rd'] / sqr_df['pct10PlusYear2Lowest3rd'],0)
sqr_df['numYear3Lowest3rdApprox'] = round(sqr_df['num10PlusYear3Lowest3rd'] / sqr_df['pct10PlusYear3Lowest3rd'],0)

back into cohort year by year 1 year 2 year 3 counts

In [39]:
sqr_df['year1Cohort'] = sqr_df['sqrYear'].astype(int) - 1
sqr_df['year2Cohort'] = sqr_df['sqrYear'].astype(int) - 2
sqr_df['year3Cohort'] = sqr_df['sqrYear'].astype(int) - 3

In [40]:
sqr_df = sqr_df.merge(closing_gap_df, right_on = ['dbn', 'sqrYear'], left_on = ['dbn', 'sqrYear'])

back into num in grad year that is lowest 3rd

In [56]:
sqr_df['numGradLowest3rdApprox'] = round(sqr_df['numGradLowest3rd'] / sqr_df['pctGradLowest3rd'],0)

In [57]:
sqr_df.head()

,dbn,sqrEnrollment,grd8Ela,grd8Math,attdRate,pctChronAbs,sqrYear,pct10PlusYear1,pct10PlusYear1Lowest3rd,pct10PlusYear2,pct10PlusYear2Lowest3rd,pct10PlusYear3,pct10PlusYear3Lowest3rd,num10PlusYear1,num10PlusYear1Lowest3rd,num10PlusYear2,num10PlusYear2Lowest3rd,num10PlusYear3,num10PlusYear3Lowest3rd,numYear1Lowest3rdApprox,numYear2Lowest3rdApprox,numYear3Lowest3rdApprox,year1Cohort,year2Cohort,year3Cohort,numCCPCILowest3rd,numCollegeReadyLowest3rd,numGradLowest3rd,numPERLowest3rd,pctCCPCILowest3rd,pctCollegeReadyLowest3rd,pctGradLowest3rd,pctPERLowest3rd,numGradLowest3rdApprox
0,01M292,255.0,2.18,2.06,0.766,0.568,2015,0.857,0.867,0.568,0.438,0.700,0.333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2013,2012,NaN,NaN,NaN,NaN,0.250,0.281,1.750,0.152,NaN
1,01M448,304.0,2.27,2.37,0.880,0.347,2015,0.889,0.800,0.872,0.720,0.742,0.533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2013,2012,NaN,NaN,NaN,NaN,0.543,0.174,3.130,0.429,NaN
2,01M450,666.0,2.66,2.63,0.938,0.165,2015,0.946,0.828,0.870,0.818,0.824,0.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2013,2012,NaN,NaN,NaN,NaN,0.368,0.316,2.737,0.625,NaN
3,01M509,363.0,2.28,2.09,0.768,0.647,2015,0.652,0.619,0.627,0.542,0.583,0.407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2013,2012,NaN,NaN,NaN,NaN,0.043,0.000,0.745,0.107,NaN
4,01M539,1735.0,3.50,3.53,0.952,0.130,2015,0.975,0.912,0.984,0.945,0.984,0.962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,2013,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
sqr_df.to_csv('data/clean/sqr.csv',index=False)

#### Location Data

In [43]:
lcgms = pd.read_excel('data/lcgms.xlsx', sheet_name='lcgms')

In [44]:
lcgms['DBN'] = lcgms['ATS System Code'].str.strip()

In [45]:
lcgms.columns

Index(['ATS System Code', 'Location Code', 'Location Name', 'BEDS Number',
       'Managed By Name', 'Location Type Description',
       'Location Category Description', 'Grades', 'Grades Final', 'Open Date',
       'Status Description', 'Building Code', 'Primary Address', 'City',
       'State Code', 'Zip', 'Borough Block Lot', 'Census Tract',
       'Community District', 'Council District', 'NTA', 'NTA_Name',
       'Principal Name', 'Principal Title', 'Principal Phone Number',
       'Fax Number', 'Geographical District Code',
       'Administrative District Code', 'Administrative District Location Code',
       'Administrative District Name', 'Superintendent',
       'Superintendent Title', 'Superintendent Location Code',
       'Community School Sup Name', 'HighSchool Network Location Code',
       'HighSchool Network Name', 'HighSchool Network Superintendent',
       'Executive Superintendent Location code',
       'Executive Superintendent Name', 'Executive Superintendent Title'

In [46]:
lcgms.groupby(['Administrative District Location Code']).count()

,ATS System Code,Location Code,Location Name,BEDS Number,Managed By Name,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,Status Description,Building Code,Primary Address,City,State Code,Zip,Borough Block Lot,Census Tract,Community District,Council District,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Name,Superintendent,Superintendent Title,Superintendent Location Code,Community School Sup Name,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,Executive Superintendent Location code,Executive Superintendent Name,Executive Superintendent Title,BCO Location Code,BCO Location Name,BCO Executive Director,BCO Executive Director Title,DBN
Administrative District Location Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DA75,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,60,61,61,60,60,61,61,61,61,61,61,61,61,61,61,61,0,0,0,0,0,0,61,61,61,61,61
DA79,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
HS01,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,0,25,25,25,0,25,25,25,25,25,25,25,25,25,25,25
HS02,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,0,39,39,39,0,39,39,39,39,39,39,39,39,39,39,39
HS03,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,0,41,41,41,0,41,41,41,41,41,41,41,41,41,41,41
HS04,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,0,47,47,47,0,47,47,47,47,47,47,47,47,47,47,47
HS05,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,46,46,47,47,0,47,47,47,0,47,47,47,47,47,47,47,47,47,47,47
HS06,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,0,26,26,26,0,26,26,26,26,26,26,26,26,26,26,26
HS07,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,58,60,60,0,60,60,60,0,60,60,60,60,60,60,60,60,60,60,60


In [47]:
lcgms.drop(columns = ['ATS System Code','Location Code', 'Location Name', 'BEDS Number',
       'Managed By Name', 'Grades', 'Grades Final', 'Open Date',
       'Status Description', 'Building Code', 'Primary Address', 'City',
       'State Code', 
       'Community District', 'Council District', 'NTA', 'NTA_Name',
       'Principal Name', 'Principal Title', 'Principal Phone Number',
       'Fax Number', 
       'Administrative District Code', 'Administrative District Location Code',
       'Administrative District Name', 'Superintendent',
       'Superintendent Title', 'Superintendent Location Code',
       'Community School Sup Name', 'HighSchool Network Location Code',
       'HighSchool Network Name', 'HighSchool Network Superintendent',
       'Executive Superintendent Location code',
       'Executive Superintendent Name', 'Executive Superintendent Title',
       'BCO Location Code', 'BCO Location Name', 'BCO Executive Director',
       'BCO Executive Director Title'], inplace=True)

In [48]:
lcgms.rename(columns={'Location Type Description':'type', 'Location Category Description':'desc',
                      'Zip':'zip', 'Borough Block Lot':'boroughBlockLot', 'Census Tract':'censusTract', 'DBN':'dbn',
                     'Geographical District Code':'geoDis'}, inplace=True)

In [49]:
lcgms.to_csv('data/clean/lcgms.csv',index=False)

#### Demographics Data

In [50]:
demo = pd.read_excel('data/demo.xlsx', sheet_name = 'School')

In [51]:
demo[['sy','sqrYear']] = demo.Year.str.split("-", expand=True)

In [52]:
demo['sqrYear'] = '20'+demo['sqrYear']

In [53]:
demo.drop(columns = ['School Name', 'Year', 
       'Grade 3K+PK (Half Day & Full Day)', 'Grade K', 'Grade 1', 'Grade 2',
       'Grade 3', 'Grade 4', 'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8',
       '# Multiple Race Categories Not Represented',
       '% Multiple Race Categories Not Represented','sy'], inplace=True)

In [54]:
demo.rename(columns={'DBN':'dbn', 'Total Enrollment':'enrollment',  'Grade 9':'gr9', 'Grade 10':'gr10', 'Grade 11':'gr11',
                     'Grade 12':'gr12', '# Female':'female', '% Female': 'pctFemale', 
                     '# Male':'male', '% Male':'pctMale', '# Asian' : 'asian', '% Asian' : 'pctAsian',
                     '# Black': 'black', '% Black':'pctBlack', '# Hispanic':'hispanic', '% Hispanic':'pctHispanic',
                     '# White':'white', '% White':'pctWhite', '# Students with Disabilities':'swd',
                     '% Students with Disabilities':'pctSwd', '# English Language Learners':'ell', 
                     '% English Language Learners':'pctEll', '# Poverty':'poverty', '% Poverty':'pctPoverty', 
                     'Economic Need Index':'econNeedInd'}, inplace=True)

In [55]:
demo.to_csv('data/clean/demo.csv',index=False)